# Gamepad Control for Cerus

The following code allows for simple remote control of the Cerus mobile robot platform via a gamepad controller.

In [93]:
#Import useful libraries
import serial
import time
import traitlets
import ipywidgets.widgets as widgets
from traitlets import HasTraits, Float
from IPython.display import display

In [94]:
#Open a serial connection with the Arduino Mega
ser = serial.Serial('COM3', 115200)

In [95]:
#Create a simple motor class with traits
class Motors(HasTraits):
    
    left = Float()
    right = Float()
       
motors = Motors()

In [97]:
#Attach game controller and link to motor sliders
#You can find the index of your controller on http://html5gamepad.com
controller = widgets.Controller(index=0)  # replace with index of your controller
display(controller)

Controller()

In [98]:
#Link the controller axes to the motors
left_link = traitlets.dlink((controller.axes[1], 'value'), (motors, 'left'))
right_link = traitlets.dlink((controller.axes[3], 'value'), (motors, 'right'))

In [101]:
#Send the move command to the Arduino via Serial when a change to the controller axes is detected
def detect(change):
    
    if change['new']:
        
        #Convert tank to twist
        linearVelocity = ((motors.left + motors.right) / 2.0) 
        angularVelocity = ((motors.right - motors.left) / 2.0) 
        
        #Create twist command
        command = f"<{linearVelocity},{angularVelocity}>"
        #print(command)
        
        ser.write(str.encode(command))       
        time.sleep(0.01)
    
motors.observe(detect, names='left')
motors.observe(detect, names='right')

In [100]:
#Create a useful emergency stop button
button = widgets.Button(description='STOP', disabled=False,  button_style='warning', tooltip='Stop motors',  icon='exclamation-triangle')
output = widgets.Output()
display(button, output)

def on_button_clicked(b):
    with output:
        command = f"<{0.0},{0.0}>"
        ser.write(str.encode(command))

button.on_click(on_button_clicked)

Button(button_style='warning', description='STOP', icon='exclamation-triangle', style=ButtonStyle(), tooltip='…

Output()

In [92]:
#Make sure to close the serial connection after you're done
ser.close()